<a href="https://colab.research.google.com/github/hosh-farms/hosh_simulator/blob/main/Sow_rotation_simulator_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np

def sow_rotation_simulator_with_stages(
    total_sows=30,
    piglets_per_cycle=8,
    sow_mortality=0.033,
    piglet_mortality=0.03,
    sow_feed_price=32,
    sow_feed_intake=2.8,
    grower_feed_price=28,
    fcr=3.2,
    final_weight=105,
    sale_price=180,
    management_fee=50000,
    management_commission=0.05,
    supervisor_salary=25000,
    worker_salary=18000,
    n_workers=2,
    shed_cost=1_000_000,
    shed_life_years=10,
    sow_cost=1_050_000,
    sow_life_years=4,
    loan_amount=0,
    interest_rate=0.1,
    loan_tenure_years=5,
    moratorium_months=0,
    medicine_cost=10000,
    electricity_cost=5000,
    land_lease=10000,
    months=60,
    abortion_rate=0.03
):
    current_sows = total_sows
    monthly_data = []

    # Depreciation
    shed_dep_rate = 1 / (shed_life_years * 12)
    sow_dep_rate = 1 / (sow_life_years * 12)

    # Loan calculation
    total_months = loan_tenure_years * 12
    monthly_rate = interest_rate / 12
    emi = 0
    if loan_amount > 0 and total_months > 0:
        emi = loan_amount * monthly_rate * (1 + monthly_rate)**total_months / ((1 + monthly_rate)**total_months - 1)
    loan_balance = loan_amount

    # Calculate sows to mate dynamically
    # Assuming average cycle length = Gestation (3.8 months) + Lactation (1.3 months) + Dry period (0.33 months) = 5.43 months
    average_cycle_length = 3.8 + 1.3 + 0.33
    sows_to_mate_per_month = total_sows / average_cycle_length

    batches = []
    ready_for_sale_batches = [] # To temporarily hold batches ready for sale

    # Calculate total capital invested
    total_capital_invested = shed_cost + sow_cost
    cumulative_cash_flow = -total_capital_invested # Start with initial investment as negative cash flow

    for month in range(1, months + 1):
        # Initialize monthly counters for sow events
        sows_mated_this_month = 0
        sows_aborted_this_month = 0
        sow_mortality_this_month = 0

        sow_feed_cost = current_sows * sow_feed_intake * 30 * sow_feed_price
        staff_cost = supervisor_salary + n_workers * worker_salary
        mgmt_fixed = management_fee

        # Mate sows based on dynamic calculation and abortion rate, starting from month 2
        if month >= 2:
            sows_to_mate = sows_to_mate_per_month # Keep as decimal
            sows_mated_this_month = sows_to_mate # Track sows mated
            sows_pregnant = sows_to_mate * (1 - abortion_rate) # Keep as decimal
            sows_aborted_this_month = sows_to_mate - sows_pregnant # Track abortions

            # Sow mortality (calculated monthly)
            sow_mortality_this_month = current_sows * (sow_mortality / 12)
            current_sows -= sow_mortality_this_month # Reduce current sows due to mortality


            if sows_pregnant > 0:
                farrow_month = month + 4
                wean_month = farrow_month + 1
                grower_start_month = wean_month
                grower_end_month = grower_start_month + 6 # Grower period is 6 months
                piglets = sows_pregnant * piglets_per_cycle * (1 - piglet_mortality)
                batches.append({
                    'batch_id': len(batches) + 1, # Assign a unique ID to each batch
                    'farrow_month': farrow_month,
                    'wean_month': wean_month,
                    'grower_start_month': grower_start_month,
                    'grower_end_month': grower_end_month,
                    'piglets': piglets,
                    'grower_feed_per_month': (piglets * fcr * final_weight) / 6 # Feed for 6 months
                })

        # Initialize counters for the current month (other than sow events)
        piglets_with_sow = 0
        current_growers = 0
        sold_pigs = 0
        revenue = 0
        grower_feed_cost = 0

        batches_to_remove_permanently = [] # Batches that are sold are removed permanently


        # Identify batches currently in different stages and sum up counts/costs
        batches_this_month = batches.copy() # Work on a copy to avoid issues while removing

        for batch in batches_this_month:
            # Piglets in lactation
            if batch['farrow_month'] <= month < batch['wean_month']:
                piglets_with_sow += batch['piglets']
            # Growers
            if batch['grower_start_month'] <= month < batch['grower_end_month']:
                current_growers += batch['piglets']
                grower_feed_cost += batch['grower_feed_per_month'] * grower_feed_price
            # Identify batches ready for sale this month
            if batch['grower_end_month'] == month:
                 ready_for_sale_batches.append(batch)


        # Implement bimonthly sale logic starting from month 13
        if month >= 13 and (month - 13) % 2 == 0 and ready_for_sale_batches:
            pigs_sold_this_period = 0
            batches_sold_this_period_ids = [] # Track IDs of batches sold

            # Find batches that were ready for sale in the last two months (current month and previous month)
            sale_period_start_month = month - 1 if month > 13 else 13 # For month 13, start from month 13
            sale_period_end_month = month

            batches_to_sell_in_this_period = [batch for batch in ready_for_sale_batches if batch['grower_end_month'] >= sale_period_start_month and batch['grower_end_month'] <= sale_period_end_month]


            for batch_to_sell in batches_to_sell_in_this_period:
                 pigs_sold_batch = batch_to_sell['piglets'] * (1 - 0.03) # Apply piglet mortality to pigs ready for sale
                 pigs_sold_this_period += pigs_sold_batch
                 batches_sold_this_period_ids.append(batch_to_sell['batch_id'])


            revenue += pigs_sold_this_period * final_weight * sale_price
            sold_pigs = pigs_sold_this_period

            # Remove batches that were sold from the ready_for_sale_batches list
            ready_for_sale_batches = [batch for batch in ready_for_sale_batches if batch['batch_id'] not in batches_sold_this_period_ids]

            # Mark the sold batches for permanent removal from the main batches list
            for batch_id in batches_sold_this_period_ids:
                for batch in batches:
                    if batch['batch_id'] == batch_id:
                         batches_to_remove_permanently.append(batch)
                         break # Found the batch, move to next id


        # Handle remaining growers in the final month
        if month == months and batches:
             pigs_sold_this_period = 0
             batches_sold_this_period_ids = []
             for batch in batches:
                 pigs_sold_batch = batch['piglets'] * (1 - 0.03) # Apply piglet mortality to all remaining pigs
                 pigs_sold_this_period += pigs_sold_batch
                 batches_sold_this_period_ids.append(batch['batch_id'])

             last_month_grower_revenue = pigs_sold_this_period * final_weight * sale_price
             revenue += last_month_grower_revenue
             sold_pigs += pigs_sold_this_period # Add to sold pigs for this month

             for batch_id in batches_sold_this_period_ids:
                 for batch in batches:
                    if batch['batch_id'] == batch_id:
                         batches_to_remove_permanently.append(batch)
                         break # Found the batch, move to next id


        # Remove sold batches permanently from the main batches list
        for batch in batches_to_remove_permanently:
            if batch in batches:
                batches.remove(batch)


        # Subtract sold pigs from the total grower count for this month AFTER calculating current_growers
        current_growers -= sold_pigs


        mgmt_comm_cost = revenue * management_commission
        other_fixed = medicine_cost + electricity_cost + land_lease
        total_operating_cost = sow_feed_cost + grower_feed_cost + staff_cost + mgmt_fixed + mgmt_comm_cost + other_fixed
        dep = (shed_cost * shed_dep_rate) + (sow_cost * sow_dep_rate)


        # Loan EMI
        if month <= moratorium_months:
            loan_payment = loan_balance * monthly_rate
        elif month <= total_months:
            interest = loan_balance * monthly_rate
            principal = emi - interest
            loan_balance -= principal
            loan_payment = emi
        else:
            loan_payment = 0

        monthly_profit = revenue - total_operating_cost - dep - loan_payment
        monthly_profit_percentage = (monthly_profit / revenue * 100) if revenue > 0 else 0

        # Calculate monthly cash flow (excluding depreciation)
        monthly_cash_flow = revenue - total_operating_cost - loan_payment
        cumulative_cash_flow += monthly_cash_flow


        monthly_data.append({
            'Month': month,
            'Piglets_Born_Alive': piglets_with_sow,
            'Growers': current_growers,
            'Sold_Pigs': sold_pigs,
            'Sows_Mated': sows_mated_this_month,
            'Revenue': round(revenue),
            'Sow_Feed_Cost': round(sow_feed_cost),
            'Grower_Feed_Cost': round(grower_feed_cost),
            'Staff_Cost': round(staff_cost),
            'Mgmt_Fee': round(mgmt_fixed),
            'Mgmt_Comm': round(mgmt_comm_cost),
            'Other_Fixed_Costs': round(other_fixed),
            'Total_Operating_Cost': round(total_operating_cost),
            'Depreciation': round(dep),
            'Loan_EMI': round(loan_payment),
            'Monthly_Profit': round(monthly_profit),
            'Monthly_Profit_%': monthly_profit_percentage,
            'Monthly_Cash_Flow': round(monthly_cash_flow),
            'Cumulative_Cash_Flow': round(cumulative_cash_flow)
        })


    df_month = pd.DataFrame(monthly_data)
    df_month['Year'] = ((df_month['Month'] - 1) // 12) + 1

    # Calculate remaining value of shed and sows
    remaining_shed_life_months = max(0, (shed_life_years * 12) - months)
    remaining_sow_life_months = max(0, (sow_life_years * 12) - months)

    remaining_shed_value = shed_cost * (remaining_shed_life_months / (shed_life_years * 12)) if (shed_life_years * 12) > 0 else 0
    remaining_sow_value = sow_cost * (remaining_sow_life_months / (sow_life_years * 12)) if (sow_life_years * 12) > 0 else 0

    remaining_asset_value = remaining_shed_value + remaining_sow_value

    # Add remaining asset value to the final cumulative cash flow
    cumulative_cash_flow_with_assets = cumulative_cash_flow + remaining_asset_value


    # Monthly summary - Apply styling after all calculations
    # Streamlit doesn't directly support pandas Styler objects for display
    # We will return the raw dataframe and handle display in Streamlit
    df_month_display = df_month.copy()


    # Yearly summary
    cols_to_sum = ['Revenue','Sow_Feed_Cost','Grower_Feed_Cost','Staff_Cost','Mgmt_Fee','Mgmt_Comm',
                   'Other_Fixed_Costs','Total_Operating_Cost','Depreciation','Loan_EMI', 'Piglets_Born_Alive', 'Sold_Pigs', 'Monthly_Profit', 'Monthly_Cash_Flow', 'Sows_Mated']

    df_year = df_month.groupby('Year')[cols_to_sum].sum()

    # Get the 'Growers' value from the last month of each year
    growers_end_of_year = df_month.groupby('Year')['Growers'].tail(1).values
    df_year['Growers'] = growers_end_of_year

    # Get the 'Cumulative_Cash_Flow' value from the last month of each year
    cumulative_cash_flow_end_of_year = df_month.groupby('Year')['Cumulative_Cash_Flow'].tail(1).values
    df_year['Cumulative_Cash_Flow'] = cumulative_cash_flow_end_of_year

    df_year.index = [f'Year {i}' for i in df_year.index]
    df_year['Cash_Profit'] = df_year['Revenue'] - df_year['Total_Operating_Cost']
    df_year['Profit_After_Dep_Loan'] = df_year['Cash_Profit'] - df_year['Depreciation'] - df_year['Loan_EMI']
    df_year['Cash_Profit_%'] = (df_year['Cash_Profit'] / df_year['Revenue'] * 100).round() if df_year['Revenue'].sum() > 0 else 0
    df_year['Profit_After_Dep_Loan_%'] = (df_year['Profit_After_Dep_Loan'] / df_year['Revenue'] * 100).round() if df_year['Revenue'].sum() > 0 else 0

    # Add Total Capital Invested column
    df_year['Total Capital Invested'] = total_capital_invested


    return df_month_display, df_year, total_capital_invested, cumulative_cash_flow_with_assets, revenue, remaining_sow_value, remaining_shed_value

st.title('House of Supreme Ham Simulator')
st.write('Use the sliders and number inputs in the sidebar to adjust the simulation parameters and see the financial results.')


st.sidebar.header('Simulation Parameters')

# Define widgets for each parameter
total_sows = st.sidebar.slider('Total Sows', min_value=10, max_value=200, value=30, step=5)
piglets_per_cycle = st.sidebar.slider('Piglets Per Cycle', min_value=5.0, max_value=15.0, value=8.0, step=0.1)
sow_mortality = st.sidebar.slider('Sow Mortality Rate', min_value=0.0, max_value=0.5, value=0.033, step=0.001, format="%.3f")
piglet_mortality = st.sidebar.slider('Piglet Mortality Rate', min_value=0.0, max_value=0.5, value=0.03, step=0.001, format="%.3f")
abortion_rate = st.sidebar.slider('Abortion Rate', min_value=0.0, max_value=0.5, value=0.03, step=0.001, format="%.3f")

st.sidebar.header('Feed and Sale Parameters')
sow_feed_price = st.sidebar.number_input('Sow Feed Price (per kg)', min_value=0, max_value=50, value=32, step=1)
sow_feed_intake = st.sidebar.slider('Sow Feed Intake (kg/day)', min_value=0.0, max_value=5.0, value=2.8, step=0.1)
grower_feed_price = st.sidebar.number_input('Grower Feed Price (per kg)', min_value=0, max_value=50, value=28, step=1)
fcr = st.sidebar.slider('Feed Conversion Ratio (FCR)', min_value=2.0, max_value=4.0, value=3.2, step=0.01, format="%.2f")
final_weight = st.sidebar.number_input('Final Weight (kg)', min_value=80, max_value=120, value=105, step=1)
sale_price = st.sidebar.number_input('Sale Price (per kg)', min_value=100, max_value=300, value=180, step=5)

st.sidebar.header('Management and Staff Costs')
management_fee = st.sidebar.number_input('Management Fee (Monthly)', min_value=0, max_value=200000, value=50000, step=10000)
management_commission = st.sidebar.slider('Management Commission Rate', min_value=0.0, max_value=0.2, value=0.05, step=0.001, format="%.3f")
supervisor_salary = st.sidebar.number_input('Supervisor Salary (Monthly)', min_value=0, max_value=50000, value=25000, step=1000)
worker_salary = st.sidebar.number_input('Worker Salary (Monthly)', min_value=0, max_value=30000, value=18000, step=1000)
n_workers = st.sidebar.number_input('Number of Workers', min_value=0, max_value=10, value=2)

st.sidebar.header('Capital Costs')
shed_cost = st.sidebar.number_input('Shed Cost', min_value=500000, max_value=5000000, value=1000000, step=100000)
shed_life_years = st.sidebar.number_input('Shed Life (Years)', min_value=1, max_value=30, value=10)
sow_cost = st.sidebar.number_input('Sow Cost (per sow)', min_value=500000, max_value=3000000, value=1050000, step=100000)
sow_life_years = st.sidebar.number_input('Sow Life (Years)', min_value=1, max_value=10, value=4)

st.sidebar.header('Loan Parameters')
loan_amount = st.sidebar.number_input('Loan Amount', min_value=0, max_value=10000000, value=0, step=100000)
interest_rate = st.sidebar.slider('Interest Rate (Annual)', min_value=0.0, max_value=0.2, value=0.1, step=0.001, format="%.3f")
loan_tenure_years = st.sidebar.number_input('Loan Tenure (Years)', min_value=1, max_value=20, value=5)
moratorium_months = st.sidebar.number_input('Moratorium Period (Months)', min_value=0, max_value=24, value=0)

st.sidebar.header('Other Fixed Costs')
medicine_cost = st.sidebar.number_input('Medicine Cost (Monthly)', min_value=0, max_value=50000, value=10000, step=1000)
electricity_cost = st.sidebar.number_input('Electricity Cost (Monthly)', min_value=0, max_value=50000, value=5000, step=1000)
land_lease = st.sidebar.number_input('Land Lease (Monthly)', min_value=0, max_value=50000, value=10000, step=1000)

st.sidebar.header('Simulation Duration')
months = st.sidebar.slider('Simulation Duration (Months)', min_value=12, max_value=120, value=60, step=12)

# Pass widget values to the simulator function
df_month, df_year, total_capital_invested, cumulative_cash_flow_with_assets, last_month_revenue, remaining_sow_value, remaining_shed_value = sow_rotation_simulator_with_stages(
    total_sows=total_sows,
    piglets_per_cycle=piglets_per_cycle,
    sow_mortality=sow_mortality,
    piglet_mortality=piglet_mortality,
    sow_feed_price=sow_feed_price,
    sow_feed_intake=sow_feed_intake,
    grower_feed_price=grower_feed_price,
    fcr=fcr,
    final_weight=final_weight,
    sale_price=sale_price,
    management_fee=management_fee,
    management_commission=management_commission,
    supervisor_salary=supervisor_salary,
    worker_salary=worker_salary,
    n_workers=n_workers,
    shed_cost=shed_cost,
    shed_life_years=shed_life_years,
    sow_cost=sow_cost,
    sow_life_years=sow_life_years,
    loan_amount=loan_amount,
    interest_rate=interest_rate,
    loan_tenure_years=loan_tenure_years,
    moratorium_months=moratorium_months,
    medicine_cost=medicine_cost,
    electricity_cost=electricity_cost,
    land_lease=land_lease,
    months=months,
    abortion_rate=abortion_rate
)

# Display results
st.header("Simulation Results")

st.subheader("Monthly Summary")
# Format currency columns for display
currency_cols_month = ['Revenue', 'Sow_Feed_Cost', 'Grower_Feed_Cost', 'Staff_Cost', 'Mgmt_Fee', 'Mgmt_Comm', 'Other_Fixed_Costs', 'Total_Operating_Cost', 'Depreciation', 'Loan_EMI', 'Monthly_Profit', 'Monthly_Cash_Flow', 'Cumulative_Cash_Flow']
for col in currency_cols_month:
    # Check if the column exists before applying formatting
    if col in df_month.columns:
        df_month[col] = df_month[col].apply(lambda x: f'₹{x:,.2f}' if pd.notnull(x) else '')
st.dataframe(df_month)

st.subheader("Yearly Summary")
# Format currency columns for display
currency_cols_year = ['Revenue', 'Sow_Feed_Cost', 'Grower_Feed_Cost', 'Staff_Cost', 'Mgmt_Fee', 'Mgmt_Comm', 'Other_Fixed_Costs', 'Total_Operating_Cost', 'Depreciation', 'Loan_EMI', 'Monthly_Profit', 'Monthly_Cash_Flow', 'Cumulative_Cash_Flow', 'Cash_Profit', 'Profit_After_Dep_Loan', 'Total Capital Invested']
for col in currency_cols_year:
     # Check if the column exists before applying formatting
    if col in df_year.columns:
        df_year[col] = df_year[col].apply(lambda x: f'₹{x:,.2f}' if pd.notnull(x) else '')

st.dataframe(df_year)

st.subheader("Financial Summary")
st.write(f"Total Capital Initially Invested: ₹{total_capital_invested:,.2f}")
st.write(f"Cumulative Cash Flow after {months} months: ₹{cumulative_cash_flow_with_assets:,.2f}")

# Calculate ROI
roi = (cumulative_cash_flow_with_assets / total_capital_invested) * 100 if total_capital_invested > 0 else 0
st.write(f"Return on Investment (ROI): {roi:.2f}%")

Overwriting app.py
